In [1]:
from train.data.supervised import MultimodalMultibandDataset
from ml4gw.distributions import PowerLaw
import torch
from lightning.pytorch.trainer import Trainer
from train.callbacks import MultimodalMultibandModelCheckpoint, MultimodalMultibandSaveAugmentedBatch
from train.model import MultimodalMultibandModule
from architectures.stackedresnets import MultimodalMultiband
from ml4gw.nn.norm import GroupNorm1DGetter
from train.metrics import TimeSlideAUROC
from matplotlib import pyplot as plt
plt.plot(0)
plt.close()
from lightning.pytorch.loggers import CSVLogger
import sys
sys.path.append('/home/seiya.tsukamoto/aframe/libs/priors/')
from priors.priors import end_o3_ratesandpops
import numpy as np
import os
import gc
from matplotlib import pyplot as plt
from collections import defaultdict
from torchmetrics.classification import BinaryAUROC

/usr/local/lib/python3.10/site-packages/lightning/fabric/__init__.py:41: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
/usr/local/lib/python3.10/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('ligo')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/site-packages/pkg_re

In [2]:
arch = MultimodalMultiband(classes = [64, 64, 64, 64], num_ifos = 2, layers = [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]], 
                           norm_layer = GroupNorm1DGetter(groups = 16), sample_rate = 2048, kernel_length = 2.75)
metric = TimeSlideAUROC(max_fpr = 1e-3, pool_length = 8, stride = 0.5)
weight_decay = 0.0
learning_rate = 0.000585
pct_lr_ramp = 0.115


batch_size = 100
batches_per_epoch = 1
num_files_per_batch = 10
chunk_size = 10000
chunks_per_epoch = 10
psd_length = 8
fftlength = None

waveform_prob = 0.277
swap_prob = 0.014
mute_prob = 0.055
left_pad = 0.25
right_pad = 0.05
snr_sampler = PowerLaw(minimum = 4, maximum = 100, index = -3)
valid_frac = 0.01
valid_stride = 0.5
num_valid_views = 2
valid_livetime = 1000



logger = CSVLogger(save_dir = '/home/seiya.tsukamoto/aframe/test', flush_logs_every_n_steps = 10)
callbacks = [MultimodalMultibandModelCheckpoint(monitor = "valid_auroc", mode = "max", save_top_k = 1, save_last = True, auto_insert_metric_name = False),
             MultimodalMultibandSaveAugmentedBatch()]
max_epochs = 1
check_val_every_n_epoch = 1
log_every_n_steps = 20



ifos = ["H1", "L1"]
train_start = 1240579783
train_stop = 1241443783
test_stop = 1244035783 
max_duration = 1000
Tb = 31536000
shifts = [0, 1]
seed = 1122

streams_per_gpu = 6

kernel_length = 2.75
prior = end_o3_ratesandpops

fftlength = None 
sample_rate = 2048
fduration = 1
highpass = 32
lowpass = 1024

inference_psd_length = 64
inference_sampling_rate = 4
inference_batch_size = 128

waveform_duration = 10
coalescence_time = 8
min_valid_duration = 10

data_dir = '/home/seiya.tsukamoto/aframe/layered/data/train'

resample_rates = [2048, 1024, 512]
high_passes = [32, 32, 32]
low_passes = [1024, 128, 64]
kernel_lengths = [0.5, 1, 2]
fft_kernel_length = 1
fft_high_pass = 32
fft_low_pass = 1024
inference_sampling_rates = [8, 4, 2]

In [3]:
module = MultimodalMultibandModule(arch = arch, metric = metric, learning_rate = learning_rate, 
                          pct_lr_ramp = pct_lr_ramp, weight_decay = weight_decay, verbose = True)
MMD = MultimodalMultibandDataset(data_dir = data_dir, ifos = ifos, sample_rate = sample_rate,
                                   valid_frac = valid_frac, batches_per_epoch = batches_per_epoch, 
                                   num_files_per_batch = num_files_per_batch, batch_size = batch_size,
                                   kernel_length = kernel_length, fduration = fduration, psd_length = psd_length, 
                                   waveform_prob = waveform_prob, max_snr = None, snr_alpha = None,
                                   left_pad = left_pad, right_pad = right_pad, fftlength = fftlength,
                                   highpass = highpass, lowpass = lowpass, snr_sampler = snr_sampler, 
                                   valid_stride = valid_stride, num_valid_views = num_valid_views, 
                                   min_valid_duration = min_valid_duration, valid_livetime = valid_livetime, 
                                   verbose = True, chunks_per_epoch = chunks_per_epoch,chunk_size = chunk_size,
                                   resample_rates = resample_rates, kernel_lengths = kernel_lengths, 
                                   high_passes = high_passes, low_passes = low_passes, 
                                   fft_kernel_length = fft_kernel_length, fft_high_pass = fft_high_pass, 
                                   fft_low_pass = fft_low_pass, inference_sampling_rates = inference_sampling_rates)
trainer = Trainer(accelerator = 'cpu', strategy = 'ddp_notebook', devices = 1, num_nodes = 1, logger = logger, 
                  callbacks = callbacks, max_epochs = 1, check_val_every_n_epoch = 1, log_every_n_steps = 20, 
                  benchmark = True, fast_dev_run = False)

2025-09-09 13:37:27,704 - root - INFO - Downloading data to /home/seiya.tsukamoto/aframe/layered/data/train


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(module, datamodule=MMD)

Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=gloo
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------



2025-09-09 13:37:30,566 - AframeDataset - INFO - Validated sample rate 2048.0
2025-09-09 13:37:30,570 - AframeDataset - INFO - Constructing sample rate dependent transforms
2025-09-09 13:37:30,668 - AframeDataset - INFO - Loading validation background data
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
2025-09-09 13:37:31,422 - AframeDataset - INFO - Constructing validation timeslides from background segments /home/seiya.tsukamoto/aframe/layered/data/train/background/background-1241437182-6601.hdf5
2025-09-09 13:37:31,454 - h5py._conv - DEBUG - Creating converter from 3 to 5
2025-09-09 13:38:09,644 - AframeDataset - INFO - Validating on 20000 waveforms
2025-09-09 13:38:09,649 - AframeDataset - INFO - Loading 20000 validation signals
2025-09-09 13:38:11,073 - AframeDataset - INFO - Initial dataloading complete
2025-09-09 13:38:11,501 - AframeModel - INFO - Scaled lr by 1 to 0.000585


Loading `train_dataloader` to estimate number of stepping batches.


2025-09-09 13:38:11,605 - AframeDataset - DEBUG - Using 6 workers for strain data loading
2025-09-09 13:38:11,689 - AframeDataset - INFO - Training on pool of 100000 waveforms. Sampling 1 batches per chunk from 10 chunks of size 10000 each epoch


/usr/local/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=20). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name   | Type                | Params
-----------------------------------------------
0 | model  | MultimodalMultiband | 6.0 M 
1 | metric | TimeSlideAUROC      | 0     
-----------------------------------------------
6.0 M     Trainable params
0         Non-trainable params
6.0 M     Total params
23.848    Total estimated model params size (MB)


2025-09-09 13:38:17,927 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
2025-09-09 13:38:17,945 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
Epoch 0: 100%|███████████████████████████████████████████| 1/1 [01:17<00:00,  0.01it/s, v_num=23, train_loss_step=0.603]
Validation: |                                                                                     | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████████████████████████████████████████████| 20/20 [05:24<00:00,  0.06it/s]2025-09-09 13:46:25,403 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
2025-09-09 13:46:25,422 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
2025-09-09 13:46:25,437 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_lo

/usr/local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('train_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.


Epoch 0: 100%|███████████████████| 1/1 [06:49<00:00,  0.00it/s, v_num=23, train_loss_step=0.603, train_loss_epoch=0.603]2025-09-09 13:46:25,776 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/checkpoints/0-1.ckpt
2025-09-09 13:46:26,052 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/checkpoints/last.ckpt
2025-09-09 13:46:26,211 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
2025-09-09 13:46:26,225 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
2025-09-09 13:46:26,238 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/metrics.csv
2025-09-09 13:46:26,242 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/metrics.csv
2025-09-09 13:46:26,246 - fsspec.local - DEBUG - open file: /home/s

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|███████████████████| 1/1 [06:49<00:00,  0.00it/s, v_num=23, train_loss_step=0.603, train_loss_epoch=0.603]
2025-09-09 13:46:26,377 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/checkpoints/0-1.ckpt
2025-09-09 13:47:37,782 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
2025-09-09 13:47:37,879 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_23/hparams.yaml
2025-09-09 13:47:37,994 - fsspec.local - DEBUG - open file: /tmp/tmp2q3l0zyx/.temp.ckpt
2025-09-09 13:47:39,485 - fsspec.local - DEBUG - open file: /tmp/tmp2q3l0zyx/.temp.ckpt


In [5]:
trainer.strategy._lightning_module = module
MMD.trainer = trainer
MMD.prepare_data()
MMD.setup(stage = 'train')

2025-09-09 01:48:40,903 - AframeDataset - INFO - Validated sample rate 2048.0
2025-09-09 01:48:40,904 - AframeDataset - INFO - Constructing sample rate dependent transforms
2025-09-09 01:48:41,090 - AframeDataset - INFO - Loading validation background data
2025-09-09 01:48:41,906 - AframeDataset - INFO - Constructing validation timeslides from background segments /home/seiya.tsukamoto/aframe/layered/data/train/background/background-1241437182-6601.hdf5
2025-09-09 01:48:41,956 - h5py._conv - DEBUG - Creating converter from 3 to 5
2025-09-09 01:49:27,866 - AframeDataset - INFO - Validating on 20000 waveforms
2025-09-09 01:49:27,871 - AframeDataset - INFO - Loading 20000 validation signals
2025-09-09 01:49:29,073 - AframeDataset - INFO - Initial dataloading complete


In [19]:
torch.serialization.add_safe_globals([ml4gw.distributions.PowerLaw])
torch.serialization.add_safe_globals([torch.distributions.transforms.AffineTransform])
torch.serialization.add_safe_globals([torch.distributions.transforms.PowerTransform])
torch.serialization.add_safe_globals([torch.distributions.uniform.Uniform])
module.__class__.load_from_checkpoint(
    '/home/seiya.tsukamoto/aframe/test/lightning_logs/version_18/checkpoints/0-1.ckpt', arch=module.model, 
    metric=module.metric)

2025-09-09 02:57:28,882 - fsspec.local - DEBUG - open file: /home/seiya.tsukamoto/aframe/test/lightning_logs/version_18/checkpoints/0-1.ckpt


MultimodalMultibandModule(
  (model): MultimodalMultiband(
    (resnets): ModuleList(
      (0-2): 3 x ResNet1D(
        (conv1): Conv1d(2, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
        (bn1): GroupNorm1D()
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (residual_layers): ModuleList(
          (0): Sequential(
            (0): BasicBlock(
              (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
              (bn1): GroupNorm1D()
              (relu): ReLU(inplace=True)
              (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
              (bn2): GroupNorm1D()
            )
            (1): BasicBlock(
              (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
              (bn1): GroupNorm1D()
              (relu): ReLU(inplace=True)
              (conv2): Conv1d(6

In [6]:
MMD.trainer.training = True
MMD.trainer.validating = False
ds = MMD.train_dataloader()
for train_batch in ds:
    train_batch = MMD.on_before_batch_transfer(train_batch, 10)
    train_batch = MMD.on_after_batch_transfer(train_batch, 10)
    break

2025-09-09 01:49:29,338 - AframeDataset - DEBUG - Using 6 workers for strain data loading
2025-09-09 01:49:29,797 - AframeDataset - INFO - Training on pool of 100000 waveforms. Sampling 2 batches per chunk from 10 chunks of size 10000 each epoch


In [7]:
X, y = train_batch

In [8]:
for batch in X:
    print(batch.size())

torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 6, 993])


In [9]:
MMD.trainer.training = False
MMD.trainer.validating = True
ds = MMD.val_dataloader()
for val_batch in ds:
    val_batch = MMD.on_before_batch_transfer(val_batch, 10)
    val_batch = MMD.on_after_batch_transfer(val_batch, 10)
    break

In [10]:
shift, X_bg, X_fg = val_batch

In [11]:
for batch in X_bg:
    print(batch.size())

for batch in X_fg:
    print(batch.size())

torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 6, 993])
torch.Size([5, 100, 2, 1024])
torch.Size([5, 100, 2, 1024])
torch.Size([5, 100, 2, 1024])
torch.Size([5, 100, 6, 993])
